# PageRank algorithm implementation

---



The PageRank algorithm was developed by Google to evaluate the importance of a web page with respect to the World Wide Web link structure. The algorithm is based on a solid mathematical model, which can be defined by means of matrix algebra. This notebook presents a basic Python implementation of the PageRank algorithm which closely follows this algebraic formulation of the problem. The results have been shown to be coherent with respect to those obtained by standard implementations.
For the referenced theoretical framework, see *Gleich, D.F. (2015). PageRank Beyond the Web. Society of Industrial and Applied Mathematics Review, Vol.57, No. 3*. **For a detailed description of the design and implementative choices, see the report**. The Jupiter notebook is used only to allow an easier interaction with the end user.

In [1]:
import numpy as np

## Graph implementation
This section is to model graphs and nodes. As a design choice, I decided to focus on the graphs and nodes aspects which are more relevant in the context of the PageRank algorithm: therefore the definition of the classes is limited to the methods and fields which I found useful for the problem, and thus they are reduced to the essential, without defining the full ADT.

In [4]:
class Node:
    """
    This class represents a node. A node has some children and parents, which we could
    link to using its methods.
    """

    def __init__(self, name):
        self.name = name
        self.children = []
        self.parents = []

    def __repr__(self):
        return "node " + self.name

    def link_child(self, new_child):
        for child in self.children:
            if(child.name == new_child.name):
                return None
        self.children.append(new_child)

    def link_parent(self, new_parent):
        for parent in self.parents:
            if(parent.name == new_parent.name):
                return None
        self.parents.append(new_parent)


class Graph:
    """
    This class models a graph for the PageRank problem. It consists essentialy of a list
    of nodes. It has methods to check the presence of a node, find a node (or create one),
    link two nodes and compute the adiacency matrix.
    """
    def __init__(self):
        self.nodes = []

    def __len__(self):
        return len(self.nodes)

    def contains(self, name):
        for node in self.nodes:
            if(node.name == name):
                return True
        return False

    def find(self, name):
        if(not self.contains(name)):
            new_node = Node(name)
            self.nodes.append(new_node)
            return new_node
        else:
            return next(node for node in self.nodes if node.name == name)

    def add_edge(self, parent, child):
        parent_node = self.find(parent)
        child_node = self.find(child)

        parent_node.link_child(child_node)
        child_node.link_parent(parent_node)

    def adjacency_matrix(self):
        A = np.zeros((len(self.nodes), len(self.nodes)))
        for i in range(len(self.nodes)):
            children = [node.name for node in self.nodes[i].children]
            row = [node.name in children for node in self.nodes]
            A[i] = row
        return A

The following cells contain utility functions for generating graphs. In particular, the procedure build_graph creates a Graph instance starting from a text file, where the graph is represented using the edge list convention.

In [5]:
def build_graph(file_name):
    """
    Creates and returns a graph object from the edge list contained in the given path.

    Parameters
    ----------
    file_name : string
        The relative path (from the dataset folder) to the file containing the graph definition.

    Returns
    -------
    graph : Graph
        The graph built from file_name.

    """
    with open(file_name) as file:
        lines = file.readlines()

    graph = Graph()

    for line in lines:
        [parent, child] = line.strip().split(',')
        graph.add_edge(parent, child)

    return graph

In [11]:
# G = build_graph("graph_5.txt") # for Colab
G = build_graph("./dataset/graph_5.txt") # in VS code / Jupyter lab
print("List of nodes:\n", G.nodes)
print("Lenght of graph:", len(G))
print("Adjacency matrix:\n", G.adjacency_matrix())

List of nodes:
 [node 2076, node 4785, node 5793, node 6338, node 9484, node 2564, node 6395, node 9994, node 5016]
Lenght of graph: 9
Adjacency matrix:
 [[0. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]


## PageRank algorithm implementation
This Section contains the actual PageRank implementation, together with the necessary helper functions.

The `P_matrix` function, as the name suggests, computes the P matrix to use in the PageRank algorithm for a given graph. The complete signature is `P_matrix(graph)`, where graph is an instance of the class `Graph`. It returns a 2 dimensional `numpy` array as a result. It is important to note that this method does not embed a policy for column completion in case of sink nodes: this means that it returns a sub-stochastic matrix. Instead, the completion policy is enforced inside the pageRank method, as it is a characteristic of the particular version of the algorithm.

In [13]:
def P_matrix(graph):
    """
    Compute the P matrix to use in the PageRank algorithm (see Langville and Gleich).
    It does not embed a policy for dealing with sink nodes (their column
    is composed of all 0s); so P is a substochastic matrix (as in a psedudo-pg problem).

    Parameters
    ----------
    graph : Graph
        the graph which P refers to.

    Returns
    -------
    P : numpyarray
        A 2 dimensional array equal to the trasposed adjacency matrix multiplied for the
        Penrose-pseudoinverse of the D matrix (see Langville and Gleich).

    """
    AT = graph.adjacency_matrix().T
    d = AT.sum(axis=0)
    d[d == 0] = 1
    return AT / d

The `pageRank` function contains the implementation of the PageRank algorithm. Therefore, this method computes the PageRank value for each node of a graph, given some parameters which will be introduced soon. The procedure applies the weakly preferential policy for sink nodes, while the personalization vector is assumed to be always a uniformly distributed one. It also allows to choose between solving the PageRank linear system directly (exact solution), or applying an iterative method using the update rule described in Gleich's paper; both approaches yield similar results in terms of accuracy, as it is discussed thoroughly inside the report.

In [14]:
def pageRank(graph, alpha=0.85, max_iterations=400, algo="iterative", rround="yes"):
    """
    Returns the PageRank value for each of the nodes of the given graph, using the
    given parameters. It applies the weakly preferential policy for sink nodes (the random
    surfer is teleported to a random node following a uniform distribution). The
    personalization vector is considered to be a uniformly distributed one.

    Parameters
    ----------
    graph : Graph (own implementation)
        The graph containing the nodes to compute the PageRank for.
    alpha : float, optional
        The damping parameter of the algorithm. The default is 0.85.
    max_iterations: int, optional
        The maximum number of iterations to do in case an iterative procedure is chosen (see
        algo parameter): when the maximum value is reached, the execution of the function stops
        and the result obtained at that point is returned. The default is 400.
    algo : string, optional
        Used to distinguish between an iterative application of the algorithm and exact one.
        For the iterative version, it applies the update rule contained in Gleich's paper until
        convergence or maximum iterations reached. The pg values vector is initialized
        as a uniform distribution over all the nodes. For the exact version, see pageRank_exact.
        The default value of the parameter is "iterative". For the exact version, use "exact".
    rround : string, optional
        String value to apply a rounding of the pg values to the first 3 decimal digits.
        The default is "yes".

    Returns
    -------
    numpy array
        An array containing the pg value for each node. Each value refers to the node in graph
        which holds the same position in the graph's node list.
    """
    if algo == "exact":
        return pageRank_exact(graph, alpha, rround)

    N = len(graph)
    P = P_matrix(graph)
    c = np.sum(P, axis=0) == 0
    v = np.repeat(1.0/N, N)  # teleportation vector
    x = v.copy()   # initialization vector
    dangling = v.copy()  # sink nodes policy vector

    threshold= 1e-16
    error = 1

    for i in range(max_iterations):
        x_old = x
        x = alpha*(P @ x_old + (np.inner(c, x_old) * dangling)) + (1-alpha) * v
        error = np.linalg.norm((x - x_old), ord=1)  # use norm 1 to convergence
        if error < threshold:
            break

    if rround == "yes":
        return np.round(x, 3)
    return x

def pageRank_exact(graph, alpha=0.85, rround="yes"):
    """
    Exact resolution for the PageRank problem. It is done by solving the linear system
    associated to the problem as shown in Gleich's paper. We still use the weakly preferential
    approach and a uniform v vector.

    Parameters
    ----------
    graph : Graph (own implementation)
        The graph containing the nodes to compute the PageRank for.
    alpha : float, optional
        The damping parameter of the algorithm. The default is 0.85.
    rround : string, optional
        String value to apply a rounding of the pg values to the first 3 decimal digits.
        The default is "yes".

    Returns
    -------
    numpy array
        An array containing the pg value for each node. Each value refers to the node in graph
        which holds the same position in the graph's node list.

    """
    P = P_matrix(graph)
    N = len(graph)
    v = np.repeat(1.0/N, N)

    # this time we need to complete P as to make it a stochastic matrix
    c = np.sum(P, axis=0) == 0
    u = np.repeat(1.0/N, N)
    P = P + np.outer(u,c)

    x = np.linalg.solve(np.eye(N,N) - alpha*P, (1-alpha)*v)

    if rround == "yes":
        return np.round(x, 3)
    return x

def pageRank_pretty_print(graph, pg_values):
    """
    Prints the page rank value of each node, explicitly associating the node IDs
    with the PR value.

    Parameters
    ----------
    graph : Graph
        The graph the PR values refer to.
    pg_values : numpy array
        The values computed over the given graph.

    Returns
    -------
    None.

    """
    print()
    print("Page rank values:")
    print("-----------------------")

    for i in range(len(graph.nodes)):
        print("Node", i+1, " [id: " + graph.nodes[i].name + "]:", pg_values[i])
    print()

Here we execute the function we defined above over the graph G.

In [16]:
# G = build_graph("graph_4.txt")
G = build_graph("./dataset/graph_4.txt")
pr_values = pageRank(G)
pageRank_pretty_print(G, pr_values)


Page rank values:
-----------------------
Node 1  [id: 1]: 0.28
Node 2  [id: 2]: 0.159
Node 3  [id: 3]: 0.139
Node 4  [id: 4]: 0.108
Node 5  [id: 5]: 0.184
Node 6  [id: 7]: 0.069
Node 7  [id: 6]: 0.061



It is possible to execute the function with a variety of configurable parameters.

In [17]:
pr_values_2 = pageRank(G, rround="no", alpha=.70, algo="exact")
pageRank_pretty_print(G, pr_values_2)


Page rank values:
-----------------------
Node 1  [id: 1]: 0.25714634997980096
Node 2  [id: 2]: 0.1530014577560855
Node 3  [id: 3]: 0.1375098525496496
Node 4  [id: 4]: 0.11149447504025642
Node 5  [id: 5]: 0.18649624677680812
Node 6  [id: 7]: 0.07885763185431499
Node 7  [id: 6]: 0.07549398604308429



## Testing the custom PageRank
Let us now consider how the PageRank implementation can be tested. The general approach used is to compare the results given by the custom implementation with those of a standard PageRank library: NetworkX. NetworkX employs by default the weakly-preferential version of the algorithm and a uniform initialization for the PageRank and personalization vectors: so it applies the exact same version of the algorithm I have used and therefore the comparison is legitimate and meaningful.
The section consists of two test cases: the first one is used to check if the output values of my implementation are within a certain tolerance limit with respect to those of NetworkX (accuracy testing), for a given execution on a graph; the second one aims to compare the average execution time of the two implementations given a graph (performance testing). To test the performance of my implementation, I compared the average execution time of the pageRank function with that of NetworkX analogous procedure, using the `timeit` Python module.  


In [18]:
import timeit
import networkx as nx

In [19]:
def nx_pagerank(file_name, alpha, rround="yes"):
    """
    This procedure applies nx's implementation on the given graph with the
    given damping parameter.

    Parameters
    ----------
    file_name : string
        The name of the file containing the desired graph to process.
    alpha : float
        The damping parameter.
    rround (optional): string
        If "yes", rounds the pg values to the first 3 decimal digits.

    Returns
    -------
    numpy.array
        An array containing the pg values for all nodes, in the order they
        appear inside the file.

    """
    with open(file_name) as f:
        lines = f.readlines()

    G = nx.DiGraph()

    for line in lines:
        t = tuple(line.strip().split(','))
        G.add_edge(*t)

    pr = nx.pagerank(G, alpha=alpha)
    if rround == "yes":
        return np.round(list(pr.values()), 3)
    else:
        return np.array(list(pr.values()))



def accuracy_test(file_name, alpha, algo):
    """
    Test procedure for comparing my algorithm results with nx's. The algorithm will pass the
    test over a certain graph if the error made over each nodes' pg value is less than
    a tenth of the expected value according to nx. Some information regarding the comparison
    between the two are displayed: the MSE and the number of nodes that hold the same position
    after being sorted according to the pr value

    Parameters
    ----------
    file_name : string
        The graph to test on.
    alpha : float
        Damping parameter.
    algo: string
        Specifies if the implementation to be used should be the iterative or the exact one;
        therefore strings "iterative" and "exact".

    Returns
    -------
    None.

    """
    print()
    print("PageRank test result for", file_name, "using", algo, "version")
    print("---------------------------------------------------------------------------------------------")

    nx_result = nx_pagerank(file_name, alpha, rround="no")
    my_result = pageRank(build_graph(file_name), alpha, algo=algo, rround="no")
    mse = np.mean((my_result-nx_result)**2)
    same_order_num = np.sum(np.argsort(nx_result) == np.argsort(my_result))

    try:
        np.testing.assert_allclose(my_result, nx_result, rtol=1e-03, err_msg="The actual PR values differ from the desired ones by more that 1/1000 of the desired value")
        error = "thousandth"
    except AssertionError as e:
        try:
            print(e)
            np.testing.assert_allclose(my_result, nx_result, rtol=1e-02, err_msg="The actual PR values differ from the desired ones by more that 1/100 of the desired value")
            error = "hundredth"
        except AssertionError as e:
            try:
                print(e)
                np.testing.assert_allclose(my_result, nx_result, rtol=1e-01, err_msg="The actual PR values differ from the desired ones by more that 1/10 of the desired value")
                error = "tenth"
            except AssertionError as e:
                print(e)
                print("The algorithm didn't pass the test for this graph.")
                print("The MSE is:", mse)
                print("The nodes are ranked in the same way for", same_order_num, "over", len(my_result), "nodes")

    print()
    print("Each pg value differs from the expected one for less than a", error, "of the expected value")
    print()
    print("The MSE is:", mse)
    print("The nodes are ranked in the same way for", same_order_num, "over", len(my_result), "nodes")
    print()

def time_test(file_name, alpha, algo, number=10, nx_time=None):
    """
    Compute the average execution time of PageRankCalculator.pageRank and .pageRank_exact given the parameters
    and compare it with the average time of NetworkX's implementation.
    The result is printed on the console.

    Parameters
    ----------
    file_name : string
        The graph to test on.
    alpha : float
        Damping parameter.
    algo: string
        Specifies if the implementation to be used should be the iterative or the exact one;
        therefore strings "iterative" and "exact"
    number: int (optional)
        The number of repeated executions for the two implementations to average on. By default it equals 10.
    nx_time: float (optional)
        The average reference time the implementations should be compared to; if not given it is computed
        inside the function. The default value is None to represent that no reference time was given to the
        function.

    Returns
    -------
    my_time: float
        The average execution time of the custom implementation of PageRank
    nx_time: float
        The average execution time over the given graph for nx's library implementation

    """
    print()
    print("Measuring average execution time of the", algo, "algorithm, over", number, "iterations on file", file_name)
    print("------------------------------------------------------------------------------------------------------------")
    print()

    globals()["file_name"] = file_name
    globals()["alpha"] = alpha
    globals()["algo"] = algo
    globals()["number"] = number

    if nx_time is None:
        nx_time = timeit.timeit("nx_pagerank(file_name, alpha, rround=\"no\")", globals=globals(), number=number)
    my_time = timeit.timeit("pageRank(build_graph(file_name), alpha, algo=algo, rround=\"no\")",
                            globals=globals(), number=number)

    comparison = "faster"
    if(my_time > nx_time):
        comparison = "slower"

    print("Average execution time for my algorithm is", my_time / number, "seconds")
    print("It is", abs(my_time - nx_time) / number, "seconds", comparison, "than NetworkX on average")
    print()

    return my_time, nx_time


We can execute the tests and see the outputs by invoking the test functions defined above.

In [21]:
# graph_name = "graph_5.txt"
graph_name = "./dataset/graph_5.txt"
accuracy_test(graph_name, .85, "iterative")
_,_ = time_test(graph_name, .85, "iterative")

# If not working, might be a librabry conflict between scipy and networkx. 
# Set networkx to <= 2.6.
# Or use Colab directly


PageRank test result for ./dataset/graph_5.txt using iterative version
---------------------------------------------------------------------------------------------


AttributeError: module 'scipy.sparse' has no attribute 'coo_array'